In [1]:
import pandas as pd

In [8]:
df = pd.read_csv(r'./dataset.csv')
print(df.head())

df2 = df[["track_id","writers","track_name","track_pos"]]
df2_dict = df2.to_dict(orient='records')

d = []

for f in df2_dict:
    d.extend(f["writers"].split("-"))

set_d = set([a.strip() for a in d])
dict_vett_writers = []
support = []
#positions = []

for writer in set_d:
    for d in df2_dict:
        if writer in [a.strip() for a in d["writers"].split("-")]:
            support.append({"name":d["track_name"],"pos":d["track_pos"]})
            #positions.append(d["track_pos"])
    dict_vett_writers.append({"writer":writer,"tracks":support})
    support = []
    #positions = []
list_edges = []
list_weights = []

for dic in dict_vett_writers:
    for dic2 in dict_vett_writers:
        for song in dic["tracks"]:
            if song in dic2["tracks"] and dic["writer"] != dic2["writer"]:
                if {"source":dic["writer"],"target":dic2["writer"]} in list_edges:
                    index = list_edges.index({"source":dic["writer"],"target":dic2["writer"]})
                    list_weights[index] += 1 + (10-song["pos"])
                else:
                    list_edges.append({"source":dic["writer"],"target":dic2["writer"]})
                    list_weights.append(1 + (10-song["pos"]))

for weight in list_edges:
    a = weight["source"]
    b = weight["target"]
    index = list_edges.index(weight)
    #print(a,b,list_weights[index])
    try:
        list_edges.remove({"source":b,"target":a})
        list_weights.remove(list_weights[index])
    except:
        pass


  track_id  track_pos                       artists   
0   1951_0          1                   Nilla Pizzi  \
1   1951_1          2  Nilla Pizzi-Achille Togliani   
2   1951_2          3              Achille Togliani   
3   1951_3          4   Achille Togliani-Duo Fasano   
4   1951_4          5  Nilla Pizzi-Achille Togliani   

                    track_name                                writers   
0             Grazie dei Fiori  G. C. Testoni- M. Panzeri-S. Seracini  \
1   La luna si veste d'argento                     V. Mascheroni-Biri   
2           Serenata a nessuno                                W. Colì   
3  Al mercato di Pizzighettone               N. Ravasini-A. Locatelli   
4            Eco tra gli abeti                E. Bonagura-C. A. Rossi   

  track_votes  
0          50  
1          30  
2          20  
3           0  
4           0  


In [32]:
writers = open("artisti.csv","w")
writers.write("Id\n")
writers.write('\n'.join([str(a["writer"]) for a in dict_vett_writers]))
writers.close()

writersTracks = open("artisti_e_canzoni.csv","w")
writersTracks.write("Writer,Songs\n")
writersTracks.write('\n'.join([str(a["writer"]) + "," + str(a["tracks"]) for a in dict_vett_writers]))
writersTracks.close()

outfile = open("edges_list.csv","w")
outfile.write("Source,Target,Weigth\n")
outfile.write('\n'.join([str(list_edges[i]["source"]) + "," + str(list_edges[i]["target"]) +","+ str(list_weights[i]) for i in range( len(list_edges))]))
outfile.close()